In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numba
import helperfunctions # this is my own python file
import seaborn as sns
from IPython.display import display, HTML



display(HTML("<style>.container { width: 90% !important; }</style>")) # Set the custom CSS to make the notebook width 90% of the window size
sns.set_context('notebook')

In [ ]:
filename_list = [r"C:\Users\pguest\Desktop\pickles\Data Taken 2024-07-22, 2024-07-22_ND4_2.avicontrol_tracer_nodrift.pkl",
                          r"C:\Users\pguest\Desktop\pickles\Data Taken 2024-07-22, 2024-07-22_ND4_1.avicontrol_tracer_nodrift.pkl"]

In [ ]:
df_list = [] # initialize list of dataframes
for filepath in filename_list:
    df_list.append(pd.read_pickle(filepath)) # read trajectory files